In [1]:
import numpy as np
from pathlib import Path
from typing import Union
from brn.mint.model import MINT, MINTSettings
from gc_preprocess_new import process_nwb_directory, process_single_nwb, split_train_test



In [2]:
from pynwb import NWBHDF5IO

file_path = "C:/Users/aqin/Downloads/generalized-click/generalized-click/DATA_P2Lab_1587.nwb"
# Open the NWB file
io = NWBHDF5IO(file_path, mode="r", load_namespaces=True)
example_data = io.read()
example_data

c:\Users\aqin\Documents\MINT\.venv\Lib\site-packages\hdmf\spec\namespace.py:583: UserWarning: Ignoring the following cached namespace(s) because another version is already loaded:
core - cached version: 2.9.0, loaded version: 2.8.0
The loaded extension(s) may not be compatible with the cached extension(s) in the file. Please check the extension documentation and ignore this warning if these versions are compatible.
  self.warn_for_ignored_namespaces(ignored_namespaces)


Data type,object
Shape,"(3,)"
Array size,24.00 bytes
Chunk shape,None
Compression,None
Compression opts,None
Compression ratio,0.5
Data type,uint8
Shape,"(25216,)"
Array size,24.62 KiB
Chunk shape,"(630,)"


In [3]:
def main(nwb_path: Union[str, Path], 
         output_dir: str,
         merge_sessions: bool = True,
         test_fraction: float = 0.2,
         verbose: bool = True):
    """Train MINT decoder on generalized click NWB data.
    
    Args:
        nwb_path: Path to NWB file or directory containing NWB files
        output_dir: Directory to save results
        merge_sessions: Whether to merge multiple sessions
        test_fraction: Fraction of data to use for testing
        verbose: Whether to print progress
    """
    # Create output directory
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    if verbose:
        print("=" * 50)
        print("MINT Decoder Training Pipeline")
        print("=" * 50)
    
    # Process NWB data
    nwb_path = Path(nwb_path)
    
    if nwb_path.is_file():
        # Single NWB file
        if verbose:
            print(f"Processing single NWB file: {nwb_path}")
        spikes, behavior, cond_ids, trial_data = process_single_nwb(nwb_path, verbose=verbose)
        
    elif nwb_path.is_dir():
        # Directory of NWB files
        if verbose:
            print(f"Processing NWB directory: {nwb_path}")
        
        sessions = process_nwb_directory(
            nwb_dir=nwb_path,
            merge_sessions=merge_sessions,
            verbose=verbose
        )
        
        # Use merged data if available, otherwise first session
        if "merged" in sessions:
            session_data = sessions["merged"]
            if len(session_data) == 4:
                spikes, behavior, cond_ids, trial_data = session_data
            else:
                spikes, behavior, cond_ids = session_data
                trial_data = []  # No trial data for merged sessions
        else:
            session_name = list(sessions.keys())[0]
            session_data = sessions[session_name]
            if len(session_data) == 4:
                spikes, behavior, cond_ids, trial_data = session_data
            else:
                spikes, behavior, cond_ids = session_data
                trial_data = []
            if verbose:
                print(f"Using data from session: {session_name}")
    else:
        raise ValueError(f"Path does not exist: {nwb_path}")
    
    if verbose:
        print(f"\nLoaded data:")
        print(f"  Spikes: {spikes.shape}")
        print(f"  Behavior: {behavior.shape}")
        print(f"  Conditions: {len(np.unique(cond_ids))} unique conditions")
    
    
    # Split into train/test
    train_spikes, train_behavior, train_conds, test_spikes, test_behavior, test_conds = split_train_test(
        spikes=spikes,
        behavior=behavior,
        condition_ids=cond_ids,
        test_fraction=test_fraction,
        verbose=verbose
    )
    return train_spikes, train_behavior, train_conds, test_spikes, test_behavior, test_conds
    


In [4]:

    # # Test decoder
    # if verbose:
    #     print(f"\nTesting decoder on {len(test_spikes)} held-out trials...")
        
    # # Predict on test data
    # X_hat, Z_hat = decoder.predict(test_spikes)
    
    # # Calculate metrics
    # neural_r2 = calculate_r2(test_spikes, X_hat)
    # behavior_r2 = calculate_r2(test_behavior, Z_hat)
    # position_r2 = calculate_r2(test_behavior[:, :2], Z_hat[:, :2])
    # velocity_r2 = calculate_r2(test_behavior[:, 2:], Z_hat[:, 2:])
    
    # if verbose:
    #     print(f"\nTest Results:")
    #     print(f"  Neural R²: {neural_r2:.3f}")
    #     print(f"  Behavior R²: {behavior_r2:.3f}")
    #     print(f"  Position R²: {position_r2:.3f}")
    #     print(f"  Velocity R²: {velocity_r2:.3f}")
    
    # # Save results with comprehensive diagnostics
    # results = {
    #     'settings': settings.__dict__,
    #     'neural_r2': neural_r2,
    #     'behavior_r2': behavior_r2,
    #     'position_r2': position_r2,
    #     'velocity_r2': velocity_r2,
    #     'train_conditions': train_conds,
    #     'test_conditions': test_conds,
    #     'test_predictions': {
    #         'neural': X_hat,
    #         'behavior': Z_hat,
    #         'true_behavior': test_behavior,
    #         'true_neural': test_spikes,
    #     },
    #     'diagnostics': {
    #         'condition_validation': condition_validation,
    #         'balancing_stats': balancing_stats,
    #         'original_data_shapes': {
    #             'spikes': spikes.shape,
    #             'behavior': behavior.shape,
    #             'conditions': len(np.unique(cond_ids))
    #         },
    #         'final_data_shapes': {
    #             'train_spikes': train_spikes.shape,
    #             'train_behavior': train_behavior.shape,
    #             'test_spikes': test_spikes.shape,
    #             'test_behavior': test_behavior.shape
    #         }
    #     }
    # }
    
    # # Add movement alignment diagnostics if available
    # if hasattr(settings, 'movement_stats'):
    #     results['diagnostics']['movement_stats'] = settings.movement_stats
    
    # np.savez(
    #     output_path / "results.npz",
    #     **results
    # )
    
    # if verbose:
    #     print(f"\nResults saved to {output_path / 'results.npz'}")
    #     print("=" * 50)
    
    # return decoder, results



In [5]:


# Run data extraction
train_spikes, train_behavior, train_conds, test_spikes, test_behavior, test_conds = main(
    nwb_path="C:/Users/aqin/Downloads/GeneralizedClick/mint_pipeline_output/nwb_files",
    output_dir="C:/Users/aqin/Downloads/GeneralizedClick/mint_pipeline_output/mint_results",
)

settings = MINTSettings(
task="mc_maze",
fs=50.0,
obs_window=500,
min_lambda=1.0,
n_rates=2000,
min_rate=0.0,
max_rate=20,
min_prob=1e-6,
interp_mode=3,
interp_max_iters=10,
interp_tolerance=0.01
)


MINT Decoder Training Pipeline
Processing NWB directory: C:\Users\aqin\Downloads\GeneralizedClick\mint_pipeline_output\nwb_files
Found 25 NWB files in C:\Users\aqin\Downloads\GeneralizedClick\mint_pipeline_output\nwb_files

First pass: extracting trial data for unified condition mapping...

Processing DATA_P2Lab_1587.nwb...
    Found 806 epochs
    Epoch types: ['Presentation' 'Reach' 'Click' 'Presentation3' 'Center' 'Presentation2'
 'FailSafe1' 'InterTrial' 'Release']
    Tasks: ['Observation_clickdrag' 'OrthoCalibration' 'DragTask_TransientDecoderHMM'
 'DragTask_SustainedDecoderHMM']
    Extracted 40 MINT-compatible trials
    Formatting arrays: 40 trials, 63 time bins, 192 neurons
    Generated 8 unique conditions:
      Condition 0: ('Observation_clickdrag', 'Reach', (np.float64(-0.14), np.float64(-0.14))) (5 trials)
      Condition 1: ('Observation_clickdrag', 'Reach', (np.float64(-0.14), np.float64(0.14))) (5 trials)
      Condition 2: ('Observation_clickdrag', 'Reach', (np.float

In [6]:

# Initialize decoder
decoder = MINT(settings)

decoder.fit(
    train_spikes,
    train_behavior,
    train_conds
)
    


init
TODO: Fit MINT model...


In [8]:
import numpy as np
import time

# ---- helpers ----
CHANNELS = ["x_pos", "y_pos", "x_vel", "y_vel"]

def r2_per_channel(y_true: np.ndarray, y_pred: np.ndarray) -> np.ndarray:
    """
    Compute R² per channel for a single trial.
    y_true, y_pred: (T, K) with K behavioral variables.
    Returns: (K,) with NaN if SS_tot == 0 or not enough valid samples.
    """
    assert y_true.ndim == 2 and y_pred.ndim == 2
    T = min(len(y_true), len(y_pred))
    if T == 0:
        return np.full(y_true.shape[1], np.nan, dtype=float)

    yt = y_true[:T]
    yp = y_pred[:T]

    K = min(yt.shape[1], yp.shape[1])
    r2 = np.full(K, np.nan, dtype=float)

    # Channel-wise R² (ignore NaNs independently per channel)
    for k in range(K):
        valid = (~np.isnan(yt[:, k])) & (~np.isnan(yp[:, k]))
        if np.count_nonzero(valid) < 2:
            continue
        yk = yt[valid, k]
        pk = yp[valid, k]
        ss_res = np.sum((yk - pk) ** 2)
        ss_tot = np.sum((yk - np.mean(yk)) ** 2)
        if ss_tot > 0:
            r2[k] = 1.0 - ss_res / ss_tot
        # else: leave as NaN (constant ground-truth channel → undefined R²)
    return r2

# ---- evaluation loop ----
count = 0
time_sum = 0.0
trial_r2s = []  # list of (4,) arrays

print("test_features:", test_spikes.shape)
print("test_behavior:", test_behavior.shape)

for idx, (feat, beh) in enumerate(zip(test_spikes, test_behavior), start=1):
    # feat: (T, N), beh: (T, 4)
    t0 = time.perf_counter()
    X_hat, Z_hat, C_hat, K_hat, Alpha_hat = decoder.predict(feat, new_segment=True)
    dt = time.perf_counter() - t0

    # Score this trial
    r2_vec = r2_per_channel(beh, Z_hat)   # (4,)
    trial_r2s.append(r2_vec)

    time_sum += dt
    count += 1
    # Print timing and R² for this trial (compact)
    r2_str = ", ".join(f"{name}: {val:.3f}" if np.isfinite(val) else f"{name}: nan"
                       for name, val in zip(CHANNELS, r2_vec))
    print(f"Trial {idx}: {dt:.3f}s | R² -> {r2_str}")

# ---- summary ----
trial_r2s = np.vstack(trial_r2s)  # (n_trials, 4)
per_feature_mean = np.nanmean(trial_r2s, axis=0)  # mean over trials, per channel
overall_mean = np.nanmean(trial_r2s)              # mean over all trials & channels
avg_time = time_sum / max(count, 1)

print("\n=== R² Summary (mean across trials) ===")
for name, val in zip(CHANNELS, per_feature_mean):
    print(f"{name:>6}: {val:.3f}")
print(f"Overall average R²: {overall_mean:.3f}")
print(f"Trial average time: {avg_time:.3f}s")


test_features: (205, 501, 192)
test_behavior: (205, 501, 4)


C:\Users\aqin\Documents\MINT\src\brn\mint\utils.py:460: RuntimeWarning: divide by zero encountered in scalar divide
  step = gradient / hessian


Trial 1: 8.425s | R² -> x_pos: -0.262, y_pos: -0.998, x_vel: -0.307, y_vel: -116.009
Trial 2: 5.699s | R² -> x_pos: -0.542, y_pos: -1.184, x_vel: -0.028, y_vel: -17.679
Trial 3: 5.530s | R² -> x_pos: -0.034, y_pos: -0.666, x_vel: -0.033, y_vel: -43.167
Trial 4: 5.514s | R² -> x_pos: -0.015, y_pos: -10.157, x_vel: -0.021, y_vel: -17.603
Trial 5: 5.597s | R² -> x_pos: -0.003, y_pos: -0.212, x_vel: -0.002, y_vel: -7.462
Trial 6: 5.778s | R² -> x_pos: -0.963, y_pos: -0.856, x_vel: -0.065, y_vel: -6.259
Trial 7: 5.658s | R² -> x_pos: -0.037, y_pos: -11589215.898, x_vel: -0.050, y_vel: -77221473.347
Trial 8: 5.492s | R² -> x_pos: -0.036, y_pos: -6801527.451, x_vel: -0.051, y_vel: -42446317.789
Trial 9: 5.514s | R² -> x_pos: -0.038, y_pos: -107823.033, x_vel: -0.053, y_vel: -718077.731
Trial 10: 5.481s | R² -> x_pos: -0.036, y_pos: -19002.723, x_vel: -0.048, y_vel: -122254.251
Trial 11: 5.556s | R² -> x_pos: -0.038, y_pos: -10854537.851, x_vel: -0.052, y_vel: -72407810.721
Trial 12: 5.804s | 

KeyboardInterrupt: 